In [ ]:
import os
import luigi
import pandas as pd

In [ ]:
class TestTaskFileOutput(luigi.Task):
    param1 = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget(
            os.path.join(self.__class__.__name__+"-{}.csv".format(self.param1))
        )

    def run(self):
        with open(self.output().path, 'w') as out:
            out.write("done")



class TestTaskDataFrameOutput(luigi.Task):
    param1 = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget(
            os.path.join(self.__class__.__name__+"-{}.csv".format(self.param1))
        )

    def run(self):
        df = pd.DataFrame([['123456', self.param1]], columns=['id','param1'])
        df.to_csv(self.output().path, encoding='utf-8', index=False)



class TestTaskCalculatedParameter(luigi.Task):
    param2 = luigi.Parameter()
    param_bool = luigi.BoolParameter(default=False)
    
    @property
    def param_bool_calculated(self):
        return 'yep' if self.param_bool else 'nah'
    
    def param_w_option(self, option):
        return "param2 is {}, param_bool_calculated is {}, option is {}".format(self.param2, self.param_bool_calculated, option)
    
    def output(self):
        return luigi.LocalTarget(
            os.path.join(self.__class__.__name__+"-{}-{}.csv".format(self.param2, self.param_bool))
        )

    def run(self):
        print(self.param_bool_calculated)
        print(self.param_w_option("my option"))
        with open(self.output().path, 'w') as out:
            out.write("done")



class TestTaskWithRequirements(luigi.Task):
    param1 = luigi.Parameter()
    param2 = luigi.Parameter()

    def requires(self):
        return [
            TestTaskFileOutput(param1=self.param1),
            TestTaskCalculatedParameter(param2=self.param2, param_bool=False)
        ]

    def output(self):
        return luigi.LocalTarget(
            os.path.join(self.__class__.__name__+"-{}-{}.csv".format(self.param1, self.param2))
        )

    def run(self):
        df = pd.DataFrame([['1', self.param1], ['2', self.param2]], columns=['id','column'])
        df.to_csv(self.output().path, encoding='utf-8', index=False)

In [ ]:
PROJECT_ID = 'projectidtest'
DATASET_ID = 'dataset'

bq_client = bigquery.BigQueryClient()

class TestRunQueryTask(bigquery.BigQueryRunQueryTask):
    query = ''' SELECT 'hello' as field1, 2 as field2 '''
    table = luigi.Parameter()
 
    def output(self):
        return bigquery.BigQueryTarget(PROJECT_ID, DATASET_ID, self.table, client=bq_client)

In [ ]:
luigi.build([TestTaskWithRequirements(param1='foo', param2='bar')])